In [ ]:
!apt-get -y install poppler-utils tesseract-ocr
!pip install pdf2image pytesseract pillow pypdf python-docx python-pptx transformers accelerate
!pip install sentence-transformers


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 41 not upgraded.
Need to get 186 kB of archives.
After this operation, 697 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.12 [186 kB]
Fetched 186 kB in 0s (436 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 121713 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.12_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.12) ...
Setting up poppler-utils (22.02.0-2ubuntu0.12) ...
Processing triggers for man-db (2.10.2-1) ...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.5/329.5 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
import uuid
import re
from dataclasses import dataclass
from pathlib import Path
from typing import List, Dict, Tuple

from pdf2image import convert_from_path
import pytesseract
from PIL import Image
from pypdf import PdfReader
import docx
from pptx import Presentation

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# ----- Load a free instruct model from Hugging Face -----
MODEL_NAME = "microsoft/Phi-3-mini-4k-instruct"  # you can change to another small instruct model if needed

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    torch_dtype="auto"
)

qa_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=False,
)

print("Model loaded.")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Model loaded.


In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np

# Small, fast, free semantic model
EMBED_MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"
embed_model = SentenceTransformer(EMBED_MODEL_NAME)

def embed_texts(texts):
    """
    Compute embeddings for a list of texts.
    Returns a numpy array of shape (n, d).
    """
    if isinstance(texts, str):
        texts = [texts]
    return np.array(embed_model.encode(texts, show_progress_bar=False))


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# ====== GLOBAL "MEMORY" STRUCTURES ======

# Full page texts
SESSION_PAGES: Dict[str, List[str]] = {}   # session_id -> [page_text_0, page_text_1, ...]

# Chunks (groups of page indices)
SESSION_CHUNKS: Dict[str, List[List[int]]] = {}  # session_id -> [[page_indices_for_chunk_0], [chunk_1], ...]

# Leaf chunk summaries
SESSION_CHUNK_SUMMARIES: Dict[str, List[str]] = {}  # session_id -> [summary_chunk_0, summary_chunk_1, ...]

# Summary tree (hierarchical)
# session_id -> List[ List[SummaryNode] ], where levels[0] = leaf level (chunks), levels[-1][0] = root
SESSION_SUMMARY_TREE: Dict[str, List[List["SummaryNode"]]] = {}


BASE_DIR = Path(".")
TEMP_DIR = BASE_DIR / "temp_pages"
TEMP_DIR.mkdir(exist_ok=True)


import string
from collections import Counter

def generate_from_model(prompt: str, max_new_tokens: int = 256) -> str:
    """
    Simple wrapper around the local qa_pipeline for text generation.
    """
    out = qa_pipeline(
        prompt,
        max_new_tokens=max_new_tokens,
        num_return_sequences=1,
        do_sample=False
    )[0]["generated_text"]

    if out.startswith(prompt):
        out = out[len(prompt):]
    return out.strip()


def basic_clean_text(text: str) -> str:
    text = text.replace("\r", " ")
    text = re.sub(r"\s+", " ", text)
    return text.strip()


def score_text_relevance(question: str, text: str) -> int:
    """
    Simple lexical relevance score: word overlap between question and text.
    """
    q_tokens = set(re.findall(r"\w+", question.lower()))
    t_tokens = set(re.findall(r"\w+", text.lower()))
    overlap = q_tokens & t_tokens
    return len(overlap)


def normalize_text(s: str) -> str:
    """Lowercase, strip, remove punctuation for fair comparison / cleaning."""
    s = s.lower().strip()
    s = s.translate(str.maketrans("", "", string.punctuation))
    s = re.sub(r"\s+", " ", s)
    return s


def clean_model_answer(ans: str) -> str:
    """
    Trim off junk like 'Document:' and extra paragraphs.
    Keep only the first main answer block.
    """
    # Cut at explicit "Document:" if present
    if "Document:" in ans:
        ans = ans.split("Document:")[0].strip()

    # Cut at obvious separators generated by model
    for sep in ["--------------------", "Question:", "Solution:"]:
        if sep in ans:
            ans = ans.split(sep)[0].strip()

    # Optionally keep only first paragraph
    parts = ans.split("\n\n")
    if parts:
        ans = parts[0].strip()

    return ans

def semantic_similarity(q_emb: np.ndarray, t_emb: np.ndarray) -> float:
    """
    Cosine similarity between two vectors.
    """
    num = np.dot(q_emb, t_emb)
    den = np.linalg.norm(q_emb) * np.linalg.norm(t_emb)
    if den == 0:
        return 0.0
    return float(num / den)


def semantic_score_text_relevance(question: str, text: str) -> float:
    """
    Semantic relevance score using MiniLM embeddings.
    """
    embs = embed_texts([question, text])
    q_emb, t_emb = embs[0], embs[1]
    return semantic_similarity(q_emb, t_emb)


def combined_score_text_relevance(question: str, text: str) -> float:
    """
    Combine lexical and semantic relevance into a single score.
    You can tune weights later; start with equal weight.
    """
    lex = score_text_relevance(question, text)     # int
    sem = semantic_score_text_relevance(question, text)  # 0..1
    return lex + sem   # simple sum; or 0.5*lex + 5*sem, etc.


@dataclass
class SummaryNode:
    """
    One node in the summary tree.

    level_idx: index of the level in tree (0 = leaf level, increasing upwards)
    node_idx: index within that level
    summary: text summary for this node
    child_nodes: indices of children in the level below (for non-leaf nodes)
    chunk_indices: which chunk indices (leaf chunks) are under this node
    """
    level_idx: int
    node_idx: int
    summary: str
    child_nodes: List[int]  # indices in the level below
    chunk_indices: List[int]  # indices into SESSION_CHUNKS[session_id]


In [ ]:
def ocr_image(image_path: Path) -> str:
    """
    Run OCR on a single image.
    """
    print(f"[TOOL] OCR on image: {image_path}")
    img = Image.open(image_path)
    text = pytesseract.image_to_string(img)
    return text


def extract_text_from_image_file(file_path: Path) -> str:
    """
    OCR directly from an image file.
    """
    print(f"[TOOL] Image file OCR: {file_path}")
    return ocr_image(file_path)


def extract_text_from_docx(file_path: Path) -> str:
    """
    Extract text from DOCX.
    """
    print(f"[TOOL] DOCX text extract: {file_path}")
    document = docx.Document(str(file_path))
    paras = [p.text for p in document.paragraphs if p.text.strip()]
    return "\n".join(paras)


def extract_text_from_pptx(file_path: Path) -> str:
    """
    Extract text from PPTX.
    """
    print(f"[TOOL] PPTX text extract: {file_path}")
    pres = Presentation(str(file_path))
    texts = []
    for slide in pres.slides:
        for shape in slide.shapes:
            if hasattr(shape, "text") and shape.text:
                texts.append(shape.text)
    return "\n".join(texts)


In [ ]:
def build_chunks_from_pages(pages: List[str], pages_per_chunk: int = 2) -> Tuple[List[str], List[List[int]]]:
    """
    Group pages into chunks of up to 'pages_per_chunk' pages each.
    Return:
      chunk_texts: List[str]  - concatenated text for each chunk
      chunk_page_indices: List[List[int]]  - page indices in each chunk
    """
    chunks = []
    chunk_page_indices = []
    n = len(pages)

    for i in range(0, n, pages_per_chunk):
        indices = list(range(i, min(i + pages_per_chunk, n)))
        texts = []
        for idx in indices:
            page_num = idx + 1
            texts.append(f"\n\n==== PAGE {page_num} ====\n\n{pages[idx]}")
        chunk_text = "".join(texts)
        chunks.append(chunk_text.strip())
        chunk_page_indices.append(indices)

    return chunks, chunk_page_indices


def summarize_chunk(chunk_text: str, chunk_idx: int, page_indices: List[int]) -> str:
    """
    Summarise a chunk (1–2 pages) into a dense, mostly extractive summary.

    IMPORTANT: When you mention deadlines, numbers, names, or exact phrases,
    copy them VERBATIM from the text. Do not change numbers.
    """
    pages_str = ", ".join(str(i + 1) for i in page_indices)
    prompt = (
        f"You are summarizing a chunk containing pages {pages_str} of a policy document.\n"
        "Write a concise summary in 3–6 bullet points.\n"
        "Your summary MUST be mostly extractive:\n"
        "- When you mention deadlines, numbers of working days, names of offices, or specific grounds,\n"
        "  you MUST copy those phrases exactly as they appear in the text (verbatim).\n"
        "- Do NOT change numbers (e.g., '15 working days' must remain '15 working days').\n"
        "- Do NOT invent additional grounds or deadlines.\n\n"
        "Chunk text:\n"
        "--------------------\n"
        f"{chunk_text}\n"
        "--------------------\n\n"
        "Summary:\n"
        "- "
    )
    return generate_from_model(prompt, max_new_tokens=256)



def summarize_pair(summary_a: str, summary_b: str) -> str:
    """
    Summarise two summaries into a parent summary, preserving key factual details.
    """
    prompt = (
        "You are merging two summaries of different parts of the same policy document.\n"
        "Write a concise summary in 3–6 bullet points.\n"
        "You MUST preserve key factual details (like deadlines, numbers of working days, offices, and named grounds)\n"
        "exactly as written in the input summaries. Do NOT change numbers or invent new ones.\n\n"
        "Summary A:\n"
        f"{summary_a}\n\n"
        "Summary B:\n"
        f"{summary_b}\n\n"
        "Merged summary:\n"
        "- "
    )
    return generate_from_model(prompt, max_new_tokens=256)



def build_summary_tree_for_session(session_id: str):
    """
    Build a hierarchical summary tree for SESSION_CHUNKS and SESSION_CHUNK_SUMMARIES.
    Tree levels are stored bottom-up:
      levels[0] = leaf level (chunk summaries)
      levels[-1][0] = root node (full document summary)
    """
    chunk_summaries = SESSION_CHUNK_SUMMARIES[session_id]
    num_chunks = len(chunk_summaries)

    # Leaf level
    leaf_level: List[SummaryNode] = []
    for i, summ in enumerate(chunk_summaries):
        node = SummaryNode(
            level_idx=0,
            node_idx=i,
            summary=summ,
            child_nodes=[],          # no children below leaves
            chunk_indices=[i]        # this leaf represents chunk i
        )
        leaf_level.append(node)

    levels: List[List[SummaryNode]] = [leaf_level]

    current_level = leaf_level
    current_level_idx = 0

    # Build upper levels until single root
    while len(current_level) > 1:
        next_level: List[SummaryNode] = []
        for i in range(0, len(current_level), 2):
            if i + 1 == len(current_level):
                # Odd node: carry up unchanged (but update level/node indices later)
                child = current_level[i]
                # Will be re-indexed later
                node = SummaryNode(
                    level_idx=current_level_idx + 1,
                    node_idx=len(next_level),
                    summary=child.summary,
                    child_nodes=[child.node_idx],
                    chunk_indices=child.chunk_indices.copy(),
                )
                next_level.append(node)
            else:
                left = current_level[i]
                right = current_level[i + 1]
                merged_summary = summarize_pair(left.summary, right.summary)
                node = SummaryNode(
                    level_idx=current_level_idx + 1,
                    node_idx=len(next_level),
                    summary=merged_summary,
                    child_nodes=[left.node_idx, right.node_idx],
                    chunk_indices=left.chunk_indices + right.chunk_indices,
                )
                next_level.append(node)

        # Re-index nodes within this level
        for new_idx, node in enumerate(next_level):
            node.node_idx = new_idx
            node.level_idx = current_level_idx + 1

        levels.append(next_level)
        current_level = next_level
        current_level_idx += 1

    SESSION_SUMMARY_TREE[session_id] = levels
    print(f"[TreeBuilder] Summary tree built with {len(levels)} levels. Root is levels[-1][0].")

def select_best_chunk_for_question(session_id: str, question: str, candidate_chunk_indices: List[int]) -> int:
    """
    Among candidate chunks (by index), choose the one whose summary
    is most relevant to the question using combined semantic + lexical score.
    """
    chunk_summaries = SESSION_CHUNK_SUMMARIES[session_id]

    best_idx = candidate_chunk_indices[0]
    best_score = -1e9

    for ci in candidate_chunk_indices:
        s = combined_score_text_relevance(question, chunk_summaries[ci])
        if s > best_score:
            best_score = s
            best_idx = ci

    return best_idx


In [ ]:
class ControllerAgent:
    """
    Ingests a document:
      - Detects file type
      - Extracts per-page text
      - Groups pages into 2-page chunks
      - Summarises each chunk
      - Builds a hierarchical summary tree by pairing chunk summaries
    """

    def ingest_document(self, file_path: str) -> str:
        path = Path(file_path)
        if not path.exists():
            raise FileNotFoundError(f"No such file: {file_path}")

        ext = path.suffix.lower()
        print(f"[ControllerAgent] Ingesting: {path.name} (ext={ext})")

        pages: List[str] = []

        if ext == ".pdf":
            # Try text layer per page
            reader = PdfReader(str(path))
            for page in reader.pages:
                page_text = page.extract_text() or ""
                pages.append(page_text)

            if any(p.strip() for p in pages):
                print("[ControllerAgent] Using direct PDF text layer (per-page).")
            else:
                print("[ControllerAgent] No text layer; using PDF->images + OCR per-page.")
                pages = []
                images = convert_from_path(str(path))
                for i, img in enumerate(images):
                    img_path = TEMP_DIR / f"{path.stem}_page_{i+1}.png"
                    img.save(img_path, "PNG")
                    txt = ocr_image(img_path)
                    pages.append(txt)

        elif ext in {".png", ".jpg", ".jpeg", ".tiff", ".bmp"}:
            txt = extract_text_from_image_file(path)
            pages = [txt]

        elif ext == ".docx":
            txt = extract_text_from_docx(path)
            pages = [txt]

        elif ext == ".pptx":
            txt = extract_text_from_pptx(path)
            pages = [txt]

        elif ext in {".txt", ".md", ".log"}:
            txt = path.read_text(encoding="utf-8", errors="ignore")
            pages = [txt]

        else:
            print("[ControllerAgent] Unknown ext; trying text read then OCR as image.")
            try:
                txt = path.read_text(encoding="utf-8", errors="ignore")
                pages = [txt]
            except Exception:
                txt = extract_text_from_image_file(path)
                pages = [txt]

        # Clean pages
        cleaned_pages = []
        for p in pages:
            cp = basic_clean_text(p)
            if cp:
                cleaned_pages.append(cp)

        if not cleaned_pages:
            raise ValueError("No text could be extracted from the document.")
        pages = cleaned_pages

        print(f"[ControllerAgent] Extracted {len(pages)} page(s) of text.")

        # Build chunks (2 pages per chunk)
        chunk_texts, chunk_page_indices = build_chunks_from_pages(pages, pages_per_chunk=2)
        print(f"[ControllerAgent] Built {len(chunk_texts)} chunk(s).")

        # Summarise each chunk
        chunk_summaries = []
        for i, (chunk_text, page_idxs) in enumerate(zip(chunk_texts, chunk_page_indices)):
            print(f"[ControllerAgent] Summarizing chunk {i+1}/{len(chunk_texts)} (pages {', '.join(str(j+1) for j in page_idxs)})...")
            s = summarize_chunk(chunk_text, i, page_idxs)
            chunk_summaries.append(s)

        # Store in session memory
        session_id = str(uuid.uuid4())
        SESSION_PAGES[session_id] = pages
        SESSION_CHUNKS[session_id] = chunk_page_indices
        SESSION_CHUNK_SUMMARIES[session_id] = chunk_summaries

        print(f"[ControllerAgent] Session created: {session_id}")
        print("[ControllerAgent] Building summary tree...")
        build_summary_tree_for_session(session_id)

        return session_id


In [ ]:
def answer_from_summaries_path(question: str, path_summaries: List[str]) -> str:
    """
    Try to answer using summaries along a single path (root -> ... -> leaf).
    If not enough, model should say EXACTLY 'NEED_FULL_CHUNK'.
    """
    blocks = []
    for level_idx, summ in enumerate(path_summaries):
        blocks.append(f"[Level {level_idx}] Summary:\n{summ}")

    prompt = (
        "You are answering ONE question about a policy document.\n"
        "You are given hierarchical summaries from general (root) to specific (leaf).\n"
        "Only use information that clearly appears in these summaries.\n"
        "If the question asks for a specific number, deadline, or named ground,\n"
        "you MUST only use it if it is explicitly present in the summaries.\n"
        "Answer in 1–2 sentences maximum.\n"
        "Do NOT invent any numbers or grounds.\n"
        "If the summaries are NOT enough to answer confidently, respond EXACTLY with:\n"
        "NEED_FULL_CHUNK\n\n"
        "Summaries along the path:\n"
        "--------------------\n"
        f"{chr(10).join(blocks)}\n"
        "--------------------\n\n"
        f"Question: {question}\n"
        "Answer:\n"
    )
    answer = generate_from_model(prompt, max_new_tokens=128)
    return clean_model_answer(answer.strip())



def answer_from_chunk_fulltext(question: str, chunk_text: str, chunk_page_indices: List[int]) -> str:
    """
    Answer using the full text of the selected chunk.
    """
    pages_str = ", ".join(str(i + 1) for i in chunk_page_indices)
    prompt = (
        "You are answering ONE question using full text from specific pages of a policy document.\n"
        "Only use information that is clearly stated in the text.\n"
        "Answer in 1–2 sentences maximum.\n"
        "Do NOT include the word 'Document'.\n"
        "Do NOT copy large portions of the text.\n"
        "Do NOT answer any additional questions.\n"
        "If you cannot find the answer, say: 'I cannot find this in these pages.'\n\n"
        f"Pages: {pages_str}\n"
        "Chunk text:\n"
        "--------------------\n"
        f"{chunk_text}\n"
        "--------------------\n\n"
        f"Question: {question}\n"
        "Answer:\n"
    )
    answer = generate_from_model(prompt, max_new_tokens=128)
    return clean_model_answer(answer.strip())



class TreeQAAgent:
    """
    Uses the hierarchical summary tree built at ingestion time.

    For each question:
      1) Start at root summary.
      2) At each level, choose the child node whose summary is most relevant to the question.
      3) Collect summaries along the path (root -> leaf).
      4) Try to answer using only those summaries.
      5) If not enough, open the most relevant underlying chunk (1–2 pages)
         and answer using full text.
    """

    def _compute_path_and_summaries(self, session_id: str, question: str):
        """Internal helper: return (path_nodes_sorted, path_summaries)."""
        levels = SESSION_SUMMARY_TREE[session_id]
        current_level_idx = len(levels) - 1
        current_node_idx = 0  # root

        path_nodes: List[SummaryNode] = []

        # Walk down the tree from root to a leaf
        while True:
            node = levels[current_level_idx][current_node_idx]
            path_nodes.append(node)

            if current_level_idx == 0:
                # leaf
                break

            child_level_idx = current_level_idx - 1
            child_level = levels[child_level_idx]

            # --- semantic + lexical scoring instead of pure lexical ---
            best_score = -1e9
            best_child_idx = node.child_nodes[0]

            for ci in node.child_nodes:
                child_summary = child_level[ci].summary
                s = combined_score_text_relevance(question, child_summary)
                if s > best_score:
                    best_score = s
                    best_child_idx = ci

            current_level_idx = child_level_idx
            current_node_idx = best_child_idx

        # order from root -> leaf by level_idx
        path_nodes_sorted = sorted(path_nodes, key=lambda n: n.level_idx)
        path_summaries = [n.summary for n in path_nodes_sorted]

        return path_nodes_sorted, path_summaries


    def answer(self, session_id: str, question: str) -> str:
        """Normal user-facing answer."""
        result = self.answer_debug(session_id, question)
        return result["answer"]

    def answer_debug(self, session_id: str, question: str) -> Dict:
        """
        Debug/telemetry answer:
          returns {
            "answer": str,
            "used_full_chunk": bool,
            "chunk_idx": Optional[int],
            "path_levels": List[int],
            "leaf_chunk_indices": List[int],
          }
        """
        if session_id not in SESSION_SUMMARY_TREE:
            raise ValueError(f"No summary tree for session: {session_id}")

        path_nodes_sorted, path_summaries = self._compute_path_and_summaries(session_id, question)

        print("[TreeQAAgent] Path levels (root to leaf):", [n.level_idx for n in path_nodes_sorted])
        print("[TreeQAAgent] Leaf chunk indices:", path_nodes_sorted[-1].chunk_indices)

        # 1) Try summary-only answer
        summary_answer = answer_from_summaries_path(question, path_summaries)
        print("[TreeQAAgent] Summary-only answer (first 200 chars):")
        print(summary_answer[:200], "...")
        used_full_chunk = False
        chunk_idx = None

        if "NEED_FULL_CHUNK" not in summary_answer:
            print("[TreeQAAgent] Answered from summaries only.")
            return {
                "answer": summary_answer,
                "used_full_chunk": used_full_chunk,
                "chunk_idx": chunk_idx,
                "path_levels": [n.level_idx for n in path_nodes_sorted],
                "leaf_chunk_indices": path_nodes_sorted[-1].chunk_indices,
            }

        # 2) Need full chunk: among all chunks under leaf, pick best one
        used_full_chunk = True
        leaf_node = path_nodes_sorted[-1]
        candidate_chunks = leaf_node.chunk_indices
        chunk_idx = select_best_chunk_for_question(session_id, question, candidate_chunks)

        pages = SESSION_PAGES[session_id]
        chunks = SESSION_CHUNKS[session_id]
        page_ids = chunks[chunk_idx]

        full_text_parts = []
        for idx in page_ids:
            full_text_parts.append(f"\n\n==== PAGE {idx+1} ====\n\n{pages[idx]}")
        chunk_text = "".join(full_text_parts)

        print(f"[TreeQAAgent] Drilling down to chunk {chunk_idx} (pages {', '.join(str(i+1) for i in page_ids)}) for full-text answer...")

        final_answer = answer_from_chunk_fulltext(question, chunk_text, page_ids)

        return {
            "answer": final_answer,
            "used_full_chunk": used_full_chunk,
            "chunk_idx": chunk_idx,
            "path_levels": [n.level_idx for n in path_nodes_sorted],
            "leaf_chunk_indices": leaf_node.chunk_indices,
        }

In [ ]:
from google.colab import files

# 1) Upload any document: PDF, DOCX, PPTX, image, TXT
uploaded = files.upload()
uploaded


Saving sample.pdf to sample.pdf


{'sample.pdf': b'%PDF-1.7\r\n%\xb5\xb5\xb5\xb5\r\n1 0 obj\r\n<</Type/Catalog/Pages 2 0 R/Lang(en) /StructTreeRoot 68 0 R/Outlines 43 0 R/MarkInfo<</Marked true>>/Metadata 476 0 R/ViewerPreferences 477 0 R>>\r\nendobj\r\n2 0 obj\r\n<</Type/Pages/Count 7/Kids[ 3 0 R 28 0 R 30 0 R 32 0 R 34 0 R 36 0 R 38 0 R] >>\r\nendobj\r\n3 0 obj\r\n<</Type/Page/Parent 2 0 R/Resources<</Font<</F1 5 0 R/F2 12 0 R/F3 14 0 R/F4 19 0 R/F5 21 0 R/F6 23 0 R>>/ExtGState<</GS10 10 0 R/GS11 11 0 R>>/ProcSet[/PDF/Text/ImageB/ImageC/ImageI] >>/MediaBox[ 0 0 595.25 842] /Contents 4 0 R/Group<</Type/Group/S/Transparency/CS/DeviceRGB>>/Tabs/S/StructParents 0>>\r\nendobj\r\n4 0 obj\r\n<</Filter/FlateDecode/Length 5218>>\r\nstream\r\nx\x9c\xc5]K\x8f\xe4\xb6\x11\xbe/0\xff\xa1\x8fv\x80\xd6\xf2-\n\x184\xb0\xf32\x12\xc4\x87\xc4\x0b\xe4`\xe4`\x04\x8eOY\xe4\xf1\xff\x81\xf0\xad"Y\xa5VOSc\x03\xeb\x99\xe9\x96\xc4b\xb1X\xfc\xaa\xea#u\xfa\xfc\xd3\xbf\x7f\xf9vz|\xfc\xfc\xe3\xf3\x1f_N\xec\xf3\x9f\x7f\xf9\xf6\xdb\xe9\xbb_\xbf\x9d\x

In [ ]:
controller = ControllerAgent()
file_name = list(uploaded.keys())[0]  # first uploaded file

session_id = controller.ingest_document(file_name)
print("Session ID:", session_id)
print("Pages:", len(SESSION_PAGES[session_id]))
print("Chunks:", len(SESSION_CHUNKS[session_id]))


[ControllerAgent] Ingesting: sample.pdf (ext=.pdf)
[ControllerAgent] Using direct PDF text layer (per-page).
[ControllerAgent] Extracted 7 page(s) of text.
[ControllerAgent] Built 4 chunk(s).
[ControllerAgent] Summarizing chunk 1/4 (pages 1, 2)...
[ControllerAgent] Summarizing chunk 2/4 (pages 3, 4)...
[ControllerAgent] Summarizing chunk 3/4 (pages 5, 6)...
[ControllerAgent] Summarizing chunk 4/4 (pages 7)...
[ControllerAgent] Session created: 5c0cb572-c5a5-4616-b822-5ee354cd5725
[ControllerAgent] Building summary tree...
[TreeBuilder] Summary tree built with 3 levels. Root is levels[-1][0].
Session ID: 5c0cb572-c5a5-4616-b822-5ee354cd5725
Pages: 7
Chunks: 4


In [ ]:
agent = TreeQAAgent()

question = "What is the main purpose of the Academic Appeals Policy?"
answer = agent.answer(session_id, question)

print("\nQ:", question)
print("A:", answer)


[TreeQAAgent] Path levels (root to leaf): [0, 1, 2]
[TreeQAAgent] Leaf chunk indices: [0, 1, 2, 3]
[TreeQAAgent] Summary-only answer (first 200 chars):
The main purpose of the Academic Appeals Policy is to outline the procedure for students to request reconsideration of academic decisions, with principles for considering such requests. ...
[TreeQAAgent] Answered from summaries only.

Q: What is the main purpose of the Academic Appeals Policy?
A: The main purpose of the Academic Appeals Policy is to outline the procedure for students to request reconsideration of academic decisions, with principles for considering such requests.


In [ ]:
# Example evaluation set structure – YOU must fill in real questions & gold answers
eval_questions = [
    {"id": 1, "question": "What is the main purpose of the Academic Appeals Policy?",
     "gold": "To set out the procedure for students to request reconsideration of decisions made about their academic studies or results, and to define the principles for considering such requests."},

    {"id": 2, "question": "Who does the Academic Appeals Policy apply to?",
     "gold": "It applies to all Queen Mary students and also to recent graduates, provided the required timeframes are observed."},

    {"id": 3, "question": "What is meant by academic judgment under this policy?",
     "gold": "Academic judgment refers to matters where the opinion of an academic expert is essential, including marks awarded, degree classification, research methodology, the quality of feedback, and course outcomes."},

    {"id": 4, "question": "What are extenuating circumstances?",
     "gold": "Circumstances beyond a student’s control that negatively affect their ability to undertake or complete an assessment."},

    {"id": 5, "question": "Within how many working days must a Formal Appeal be submitted?",
     "gold": "15 working days."},

    {"id": 6, "question": "What are the two grounds on which an appeal will be considered?",
     "gold": "Procedural error and extenuating circumstances."},

    {"id": 7, "question": "Who carries the burden of proof in a Formal Appeal?",
     "gold": "The student carries the burden of proof."},

    {"id": 8, "question": "Within how many working days must a Final Review be submitted?",
     "gold": "10 working days."},

    {"id": 9, "question": "Who can students complain to after completion of internal procedures?",
     "gold": "The Office of the Independent Adjudicator for Higher Education."}
]

len(eval_questions)


9

In [ ]:
import string
from collections import Counter

def normalize_text(s: str) -> str:
    """Lowercase, strip, remove punctuation for fair comparison."""
    s = s.lower().strip()
    s = s.translate(str.maketrans("", "", string.punctuation))
    s = re.sub(r"\s+", " ", s)
    return s


def exact_match(pred: str, gold: str) -> int:
    """1 if normalized prediction == normalized gold, else 0."""
    return int(normalize_text(pred) == normalize_text(gold))


def f1_score(pred: str, gold: str) -> float:
    """Token-level F1 score between pred and gold."""
    pred_tokens = normalize_text(pred).split()
    gold_tokens = normalize_text(gold).split()

    if not pred_tokens and not gold_tokens:
        return 1.0
    if not pred_tokens or not gold_tokens:
        return 0.0

    common = Counter(pred_tokens) & Counter(gold_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0.0

    precision = num_same / len(pred_tokens)
    recall = num_same / len(gold_tokens)
    return 2 * precision * recall / (precision + recall)


In [ ]:
import time
import numpy as np

def evaluate_tree_agent(session_id: str, eval_questions: List[Dict], top_k_display: int = 5):
    agent = TreeQAAgent()

    em_scores = []
    f1_scores = []
    used_full_list = []
    times = []

    results = []

    for item in eval_questions:
        qid = item["id"]
        question = item["question"]
        gold = item["gold"]

        print(f"\n=== Q{qid}: {question} ===")
        start = time.time()
        res = agent.answer_debug(session_id, question)
        end = time.time()

        pred = res["answer"]
        used_full = res["used_full_chunk"]

        em = exact_match(pred, gold)
        f1 = f1_score(pred, gold)

        em_scores.append(em)
        f1_scores.append(f1)
        used_full_list.append(used_full)
        times.append(end - start)

        print(f"Gold: {gold}")
        print(f"Pred: {pred}")
        print(f"EM={em}, F1={f1:.3f}, used_full_chunk={used_full}, time={end-start:.2f}s")

        results.append({
            "id": qid,
            "question": question,
            "gold": gold,
            "pred": pred,
            "em": em,
            "f1": f1,
            "used_full_chunk": used_full,
            "time": end - start,
            "path_levels": res["path_levels"],
            "leaf_chunk_indices": res["leaf_chunk_indices"],
            "chunk_idx": res["chunk_idx"],
        })

    # Aggregate metrics
    mean_em = float(np.mean(em_scores)) if em_scores else 0.0
    mean_f1 = float(np.mean(f1_scores)) if f1_scores else 0.0
    mean_time = float(np.mean(times)) if times else 0.0
    full_rate = float(np.mean(used_full_list)) if used_full_list else 0.0

    print("\n===== EVALUATION SUMMARY =====")
    print(f"# questions: {len(eval_questions)}")
    print(f"Exact Match (avg): {mean_em:.3f}")
    print(f"F1 score (avg):    {mean_f1:.3f}")
    print(f"Avg time / Q:      {mean_time:.2f}s")
    print(f"% using full chunk: {full_rate*100:.1f}% (rest answered from summaries only)")

    return {
        "results": results,
        "mean_em": mean_em,
        "mean_f1": mean_f1,
        "mean_time": mean_time,
        "full_chunk_rate": full_rate,
    }


In [ ]:
summary = evaluate_tree_agent(session_id, eval_questions)
summary



=== Q1: What is the main purpose of the Academic Appeals Policy? ===
[TreeQAAgent] Path levels (root to leaf): [0, 1, 2]
[TreeQAAgent] Leaf chunk indices: [0, 1, 2, 3]
[TreeQAAgent] Summary-only answer (first 200 chars):
The main purpose of the Academic Appeals Policy is to outline the procedure for students to request reconsideration of academic decisions, with principles for considering such requests. ...
[TreeQAAgent] Answered from summaries only.
Gold: To set out the procedure for students to request reconsideration of decisions made about their academic studies or results, and to define the principles for considering such requests.
Pred: The main purpose of the Academic Appeals Policy is to outline the procedure for students to request reconsideration of academic decisions, with principles for considering such requests.
EM=0, F1=0.618, used_full_chunk=False, time=10.43s

=== Q2: Who does the Academic Appeals Policy apply to? ===
[TreeQAAgent] Path levels (root to leaf): [0, 1, 2]

{'results': [{'id': 1,
   'question': 'What is the main purpose of the Academic Appeals Policy?',
   'gold': 'To set out the procedure for students to request reconsideration of decisions made about their academic studies or results, and to define the principles for considering such requests.',
   'pred': 'The main purpose of the Academic Appeals Policy is to outline the procedure for students to request reconsideration of academic decisions, with principles for considering such requests.',
   'em': 0,
   'f1': 0.6181818181818182,
   'used_full_chunk': False,
   'time': 10.427090406417847,
   'path_levels': [0, 1, 2],
   'leaf_chunk_indices': [0, 1, 2, 3],
   'chunk_idx': None},
  {'id': 2,
   'question': 'Who does the Academic Appeals Policy apply to?',
   'gold': 'It applies to all Queen Mary students and also to recent graduates, provided the required timeframes are observed.',
   'pred': 'The Academic Appeals Policy applies to students at Queen Mary University of London who wish to